In [34]:
from src.prepare import telco_model_prep
import pandas as pd
import numpy as np

#Estimator that has built-in cross-validation capabilities to automatically select the best hyper-parameters
#Roughly equivalent to GridSearchCV(Estimator(), ...).
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

In [35]:
telco_df = telco_model_prep()

In [36]:
telco_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   SeniorCitizen                          7043 non-null   int64  
 1   tenure                                 7043 non-null   int64  
 2   MonthlyCharges                         7043 non-null   float64
 3   TotalCharges                           7043 non-null   float64
 4   gender_Male                            7043 non-null   uint8  
 5   Partner_Yes                            7043 non-null   uint8  
 6   Dependents_Yes                         7043 non-null   uint8  
 7   PhoneService_Yes                       7043 non-null   uint8  
 8   MultipleLines_Yes                      7043 non-null   uint8  
 9   InternetService_Fiber_optic            7043 non-null   uint8  
 10  InternetService_No                     7043 non-null   uint8  
 11  Onli

In [37]:
telco_df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_Yes,InternetService_Fiber_optic,...,TechSupport_Yes,StreamingTV_Yes,StreamingMovies_Yes,Contract_One_year,Contract_Two_year,PaperlessBilling_Yes,PaymentMethod_Credit_card_(automatic),PaymentMethod_Electronic_check,PaymentMethod_Mailed_check,Churn_Yes
0,0,1,29.85,29.85,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1,0,34,56.95,1889.50,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,2,53.85,108.15,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,1
3,0,45,42.30,1840.75,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,0,2,70.70,151.65,0,0,0,1,0,1,...,0,0,0,0,0,1,0,1,0,1


In [38]:
churn = np.array(telco_df.pop("Churn_Yes"))

In [76]:
#algorithm will not converge without this step
telco_df = scale(telco_df) #indpendently standardizes each feature (mu=0, var=1)

In [77]:
X_train, X_test, y_train, y_test = train_test_split(telco_df, churn, test_size=0.2, random_state=36, stratify=churn)

In [78]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5634, 23), (1409, 23), (5634,), (1409,))

In [79]:
X_train[0], y_train[0]

(array([-0.57379594,  0.93061513, -0.17431772,  0.52960364,  0.79083317,
         0.99388443,  1.43096246,  0.13753189, -1.09676081, -1.08803101,
        -0.54972032, -0.80838305,  1.4182332 , -0.96295723,  1.62618615,
         1.33141211, -1.05318813, -0.64407012, -0.7178877 , -1.35338663,
         1.81559555, -0.79000506, -0.5776673 ]),
 0)

In [82]:
clf = LogisticRegressionCV(cv=5, scoring='accuracy', random_state=36).fit(X_train, y_train)

In [83]:
clf.score(X_test, y_test)

0.8090844570617459